In [1]:
# -*- coding: utf-8 -*-
import os,sys
import pandas as pd
import numpy as np
from collections import Counter
import nltk
import string
from nltk.corpus import stopwords
import nltk.tokenize as word_tokenize
# from nltk.stem import SnowballStemmer

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("russian") # Choose a language

FIN      = 'data/data.csv'
FOUT     = 'data/data_clear.csv'
FOUTSPAM = 'data/data_spam.csv'

In [2]:
df = pd.read_csv(FIN) #FULL dirty dataset

## 1. Очистим датасет от дубликатов

In [3]:
def tokenize_me(file_text):
    tokens = nltk.word_tokenize(file_text, language='english')
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на','-','br'])
    tokens = [i for i in tokens if ( i not in stop_words )]

    return tokens
def decode(a, encoding="utf8"):
    if isinstance(a, bytes):
        return a.decode(encoding)
    else:
        return a

In [4]:
def get_shingles_lst(s):
    s = decode(s).replace('http://','').replace('https://','').replace('<br>','')
    s = s.replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')
    return list(filter(lambda x:not u'#' in x, s))

N = 4          # shingle len
all_shingles = Counter()
for S in df['data'].get_values()[2:]:
    words = get_shingles_lst(S)
    #   filter(lambda x:len(x)>2,decode(S).lower().replace('href=http','').replace('<br>','').split(' '))
    words = list(filter(lambda x:len(x)>2 and x[0]!=u'#', words))
    shingles_temp = []
    for c in range(max(0,len(words)-N)):
        temp = " ".join([words[c+c1] for c1 in range(N)])#words[c]+' '+words[c+1]+' '+words[c+2]

        shingles_temp.append(temp)
    
    for hsh in set(shingles_temp):
        all_shingles[hsh] += 1

all_shingles['- - -'] = 5

In [5]:
MAX_LENGTH = 1
bad_posts = []
max_print = 0
c_bad = 0
for S in df['data'].get_values():
    if max_print>10:
        break
    temp_len = 0
    temp = []
    words = get_shingles_lst(S)
    for c in range(max(0,len(words)-N)):
        temp = " ".join([words[c+c1] for c1 in range(N)])
        if all_shingles[temp] > 1:# or u'vk.com/ap1klsjda2p' in S:
            temp_len+=1
    if temp_len>1:
        bad_posts.append(1)
    else:
        if u'vk.com/app' in S:
            c_bad+=1
        bad_posts.append(0)
print(u'Найдено дубликатов:')
print(u"По шинглам\t\t\t\t"+str(sum(bad_posts)))
print(u"По простой ключевой фразе vk.com/app\t"+str(c_bad))

Найдено дубликатов:
По шинглам				230
По простой ключевой фразе vk.com/app	447


In [6]:
a = set(df['data'])
b = list(df['data'])
bad_dublicate_rows = []
for txt in a:
    if b.count(txt) != 1:
        lbl = df[df['data']==txt]['label']
        bad_single = False
        for i in list(lbl)[1:]:
            if i != list(lbl)[0]:
                bad_single = True
        if bad_single:
            bad_dublicate_rows.extend(lbl.index)
        else:
            bad_dublicate_rows.extend(lbl[1:].index)
print("Чистые дупликаты:\t"+str(len(df.iloc[bad_dublicate_rows])))

Чистые дупликаты:	151


In [7]:
def in_post(post, lst = u'vk.com/app\thttp://vk.com/indykot\t очков на '):
    bad = False
    for i in lst.split('\t'):
        if i in post:
            bad = True
    return bad

bad_posts_index = bad_dublicate_rows
for c,S in enumerate(df['data'].get_values()):
    if in_post(S):
#     bad_posts[c]==1 or u'vk.com/app' in S or u'http://vk.com/indykot' in S or u' очков на ' in S: #or big_post(S): delete long posts
        bad_posts_index.append(c)


In [8]:
row_mask = df.index.isin(bad_posts_index)
df_spam = df[row_mask]  # Спам

row_mask = df.index.isin(list(set(range(len(df)))-set(bad_posts_index)))
df_all = df[row_mask]   # Чистые посты

## Сохранить полученные данные

In [9]:
df_spam.to_csv(FOUTSPAM,     index = False)
df_all.to_csv( FOUT, index = False)

## Небольшая статистика по удаленным дубликатам

In [10]:
def big_post(s, max_len = 128):
    return s.replace('  ',' ').replace('  ',' ').replace('  ',' ').count(u' ')>128

print(u'В спаме:   '+str(len(df_spam)))
for i in [-1,0,1]:
    print(u'\t'+str(i)+u'\t'+str(list(df_spam['label'].get_values()).count(i)))

print(u'В выборке: '+str(len(df_all)))
for i in [-1,0,1]:
    print(u'\t'+str(i)+u'\t'+str(list(df_all['label'].get_values()).count(i)))
    
print(u'===\nДлинных постов(не в спаме): ')
len_all = 0
for ii in [-1,0,1]:
    len_of_words=0
    for i in df_all[df['label']==ii]['data']:
        if big_post(i):
            len_of_words+=1
    len_all += len_of_words      
    print(u'\t'+str(ii)+u'\t'+str(len_of_words)+'\t'+str(len_of_words*100.0/len(df_all[df['label']==ii]['data']))+'%')
print(u'  Sum '+str(len_all))

В спаме:   628
	-1	11
	0	491
	1	126
В выборке: 4247
	-1	339
	0	2620
	1	1288
===
Длинных постов(не в спаме): 
	-1	12	3.5398230088495577%
	0	188	7.175572519083969%
	1	17	1.3198757763975155%
  Sum 217


/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
